## Yellow Pages Data Scrapping

> ### Inputs.  

In [1]:
search_subject = "new Car Dealers"
search_location = 'california'
pages_to_scrape = 5
site_name = "Yellow Pages"
record_file = "no"
url ="https://www.yellowpages.com/search?search_terms=new+car+dealer&geo_location_terms=CA"

> ### Dependencies.

In [2]:
from bs4 import BeautifulSoup
from lxml import etree
import httpx                         
from fake_useragent import UserAgent
import datetime
import time
from time import sleep
import pandas as pd 
import numpy as np
import json
import os
from tqdm import tqdm
import re

In [3]:
next_page = None
'''the next page url, it's value is None for the first page scraped.'''

page = 1
'''the page number to be scraped.'''

breaker = False 
'''json values in the controls.json file which terminates the scraping process if it's values is .'''

results_scraped = 0
'''counts the number of leads scraped.'''

site = url
'''the url of the page to be scraped, it's the original url for the first page.'''

date_time = str( f'{datetime.datetime.now()}')[:f'{datetime.datetime.now()}'.index('.')].replace(':','.')
'''the date and time in which the scraping process is initialized ,
it will be add to the names of both the txt tracker file &
the csv files which contains the scarped data'''

primary_stage = True
'''dictates the starting stage of the scraping process, if it's false then it will start from the secondary stage'''

secondary_stage = True
'''dictates the starting stage of the scraping process, if it's false then 
it will join the already saved PRIMARY & SECONDARY together.'''

result_list =[]

Checking If There Was A File Which Contains The Record Of A Previous Uncompleted Scraping Process And Use It Complete Complete It, If It'S A New Scrape The Create A New Txt File To Record It'S Progress

In [4]:
if record_file.lower() != 'no':
    txt_tracker = record_file
else:
    txt_tracker = f'tracker {search_subject} in {search_location} at {date_time}.txt'
    first_creation = open('outputs/' + txt_tracker, "w")
    first_creation.close()
try:
    with open('outputs/'+ txt_tracker , 'r') as file:
        file = file.readlines()
        '''an txt file which records the progress of the scraping process'''
        
        for line in file:
            if 'https://' in line:           
                site = line.strip("/n")
                
            elif 'primary.csv' in line:
                I = int(line[line.index(search_location):].split(' ')[1].strip("/n"))
                
            elif 'PRIMARY' in line:
                primary_stage = False 
                               
            elif 'SECONDARY' in line:
                secondary_stage = False
  
    save_index = I # they are equal as every page is save in it's own csv file
    
except:
    save_index = 0
    '''the number of the last saved primary or primary file
    (incase of building upon old uncompleted scaping ), it's zero for 
    '''
    I = 0
    '''the number of the last page scraped.'''

> ### Primary Scraper Code Cell.

In [5]:
if primary_stage == True:

    ua = UserAgent().random

    # getting the request headers for the json file 'controls.json'
    J = open('controls.json', 'r')
    data = json.load(J)
    header = data["headers"]

    # getting the no.of available results for a search subject   
    session = httpx.Client()
    session.headers = header
    webpage = session.get(site)
    path_soup = BeautifulSoup(webpage.content,"html.parser")
    path = etree.HTML(str(path_soup))
    soup = BeautifulSoup(webpage.text,"html.parser")


    results_available_E = (path.xpath('//*[@id="main-content"]//div[@class="pagination"]/span'))[0].text
    results_available = int(results_available_E.split('Showing ')[1].split(" of")[0].split("-")[1])
    print(results_available, "result page available.")


    try:
        if pages_to_scrape > results_available:
            pages_to_scrape = results_available  
        pages_to_scrape =range(1,pages_to_scrape+1)
    except:
        pages_to_scrape = pages_to_scrape[I:]
    
    
    for Page in tqdm(pages_to_scrape, desc='Primary Stage',unit= "page", ncols = 110, colour= '#0A9092'):
        
        # insuring the scrape of the fist page which used to get the available result
        if next_page != None:
            
            while breaker == False: #if breaker tuned to true break the wile loop 'stop scraping'
        
                ua = UserAgent()
                J = open('controls.json', 'r')
                data = json.load(J)
                headers = data["headers"]
                #nap: time in seconds which control the sleeping interval of request making incase of 
                # unsuccessful connection to the site servers.
                nap = data["nap"]
                breaker = data["break"] 
                
                try: 
                    session = httpx.Client()
                    session.headers = header
                    webpage = session.get(site)
                    path_soup = BeautifulSoup(webpage.content,"html.parser")
                    path = etree.HTML(str(path_soup))
                    soup = BeautifulSoup(webpage.text,"html.parser") 
                    break
                
                except:
                    J = open('controls.json', 'r')
                    data = json.load(J)
                    headers = data["headers"]
                    nap = data["nap"]
                    breaker = data["break"] 
                    sleep(nap)  
                    
                    
        if breaker == True: #if breaker tuned to true break the for loop 'stop scraping'
            break
            
        primary_info = soup.select('div[class="info-section info-primary"]')# REMOVE THE S IN ELEMENTS AFTER DONE TESTING
        secondary_info = soup.select('div[class="info-section info-secondary"]') # searches for primary info container
        
        for P,S in zip(primary_info, secondary_info): # iterates through the results
                    
            business_E = P.find(class_='business-name')
            
            business = business_E.text # business name
            
            business_profile = "https://www.yellowpages.com"+business_E["href"] # the business's profile 
            
            try:
                address_E = S.find(class_="adr")#.text # gets the business location
                address = address_E.find(class_= 'street-address').text+ ", " + address_E.find(class_= 'locality').text
            except:
                address = "Not Available"
            
            try:
                phone1 = S.select("div[class='phones phone primary']")[0].text # gets the business phone number
            except:
                phone1 = "Not Listed"  
                    
            try:
                site = P.select('a[class="track-visit-website"]')[0]['href']# business website   
            except:
                site = "Not Available"      

            try:
                rating_E = P.select('div.ratings > a > div')[0]['class'][1:]
                
                if 'one' in rating_E[0]:
                    rating = 1                
                elif 'two' in rating_E[0]:
                    rating = 2
                elif 'three' in rating_E[0]:
                    rating = 3
                elif 'four' in rating_E[0]:
                    rating = 4
                else:
                    rating = 5
                    
                if len(rating_E)==2:
                    rating += 0.5                 
            except:
                rating = 0.0
            
            try:
                review_count = int(P.select('div.ratings > a > span')[0].text[1:-1])
            except:
                review_count = 0
            
            result = {"Business Name": business,
                    "Profile": business_profile,
                    "Phone Number" :phone1,
                    "Address": address,
                    "Website": site, 
                    "Rating": rating, 
                    "Review Count": review_count
                    }
            
            for key, value in result.items():
                if value == '':
                    result[key] = "Not Listed"
                
            result_list.append(result)
        
        # if len(result_list) >= 100: # saving the outputs to a csv file every 100 page
        results_scraped += len(result_list)
        save_index += 1
        
        m_df = pd.DataFrame(result_list)
        m_df.to_csv('outputs/' + f"{site_name}  {search_subject} in {search_location} {save_index} primary.csv", index= False)
        
        result_list = []

        with open('outputs/' + txt_tracker,'a') as file:
            file.write(
                '\n' +
                f"{site_name}  {search_subject} in {search_location} {save_index} primary.csv"+
                '\n'+
                F'{site}'
            )   
 
        page +=1
            
        try:
            previous_page = site
            next_page = f'&page={page}'
            site = url+ next_page
            next = True
            try:
                site = site[:site.index('&User-Agent')]+site[site.index('&page'):]
            except:
                pass    
        except:
            print("scrapping is interrupted >>LAST PAGE ERROR<< !!")
            break

        sleep(0.5)
    file.close()
else:
    print('primary stage is already completed.')

30 result page available.


Primary Stage: 100%|██████████████████████████████████████████████████████████| 5/5 [00:20<00:00,  4.19s/page]


Reading The Individual Primary Files To Build One Data Frame And Save It As One File

In [6]:
try:
    with open('outputs/' + txt_tracker,'r') as file:
        contents = file.read()
        P_csv_file = re.search('\w+PRIMARY\w+.csv',contents.replace(' ', '_'))[0].replace('_', ' ')
        m_df = pd.read_csv('outputs/' + P_csv_file)
        
except:    
    with open('outputs/' + txt_tracker,'r') as file:
        for line in file: 
            if f' {search_location} 1 primary.csv' in line:
                m_df0 = pd.read_csv('outputs/' + str(line).strip('\n'))
                
            elif 'primary.csv' in line: 
                m_df = pd.concat([m_df0, pd.read_csv('outputs/' + str(line).strip('\n'))], axis=0, ignore_index= True)
                m_df0 = m_df
  
main_df = m_df

In [7]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Business Name  151 non-null    object 
 1   Profile        151 non-null    object 
 2   Phone Number   151 non-null    object 
 3   Address        151 non-null    object 
 4   Website        151 non-null    object 
 5   Rating         151 non-null    float64
 6   Review Count   151 non-null    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 8.4+ KB


In [8]:
main_df.tail()

,Business Name,Profile,Phone Number,Address,Website,Rating,Review Count
146,Reliable Transportation,https://www.yellowpages.com/vernon-ca/mip/reli...,(323) 813-0120,"2043 Ross St, Vernon, CA 90058",http://www.reliable-transportation.com,0.0,0
147,Khm Inc,https://www.yellowpages.com/los-angeles-ca/mip...,(213) 388-2020,"3071 W Olympic Blvd, Los Angeles, CA 90006",Not Available,0.0,0
148,Electric Car Sales & Service,https://www.yellowpages.com/long-beach-ca/mip/...,(562) 436-6241,"3850 Cherry Ave, Long Beach, CA 90807",http://electriccarsalesandservice.com,0.0,0
149,Professional Leasing,https://www.yellowpages.com/redwood-city-ca/mi...,(650) 361-8838,"260 Main St, Redwood City, CA 94063",Not Available,0.0,0
150,De Anza Porsche,https://www.yellowpages.com/riverside-ca/mip/d...,(909) 824-2150,"3150 Adams St, Riverside, CA 92504",Not Available,0.0,0


In [9]:
main_df.to_csv('outputs/' + f"{site_name} PRIMARY {search_subject} in {search_location}.csv", index= False)

with open('outputs/' + txt_tracker,'a') as file:         
    file.write('\n')
    file.write (f"{site_name} PRIMARY {search_subject} in {search_location}.csv") 

In [10]:
# deleting the individual primary files after saving the concatenated PRIMARY file
files = os.listdir('outputs/')
for file in files:
    if 'primary' in file:
        os.remove('outputs/' + file)

In [11]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Business Name  151 non-null    object 
 1   Profile        151 non-null    object 
 2   Phone Number   151 non-null    object 
 3   Address        151 non-null    object 
 4   Website        151 non-null    object 
 5   Rating         151 non-null    float64
 6   Review Count   151 non-null    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 8.4+ KB


In [12]:
# separating the main list and the scrapped page variable to make prevent data loss when relaunching the scraper in 
# case of connection interruption
contacts = []

profile_no = 0
'''counts the number of leads scraped.'''

breaker = False
'''json values in the controls.json file which terminates the scraping process if it's values is .'''

"json values in the controls.json file which terminates the scraping process if it's values is ."

Checking The Txt File If There Are Secondary Files Already Scraped To Continue Upon It

In [13]:
try:
    file = open('outputs/' + txt_tracker, 'r').readlines()
    for line in file:
        if 'secondary.csv' not in line and not 'https://www' in line:
            I = int(line.strip("/n"))
            """the index of the last scraped profile url in it's secondary df"""
            
        elif 'secondary.csv'  in line:
            save_index = int(line[line.index(search_location):].split(' ')[1].strip("/n"))
            """the number of the last saved secondary csv file"""
except:
    I = 0 
    save_index = 0
    

> ### Secondary Stage

In [14]:
if secondary_stage == True:
    
    for profile in tqdm(list(main_df["Profile"][I:]), desc='Secondary Stage',unit= "profile", ncols = 110, colour= '#0A9092'):
    
        while True:
            ua = UserAgent()
            J = open('controls.json', 'r')
            data = json.load(J)
            headers = data["headers"]
            nap = data["nap"]
            break_ = data["break"]
            
            try:
                
                session = httpx.Client()
                session.headers = headers
                webpage = session.get(profile)
                path_soup = BeautifulSoup(webpage.content,"html.parser")
                path = etree.HTML(str(path_soup))
                soup = BeautifulSoup(webpage.text,"html.parser")
                break
                
            except:
                ua = UserAgent().random
                sleep(nap)
                
        if breaker == True:
            break
        
    # catagories
        try:
            catagories_E = soup.select('dd.categories > div.categories > a')
            catagories = ""
            for cat in catagories_E:
                catagories+= cat.text + ", "
            catagories = catagories.strip(', ')
        except:
            catagories= "Not Listed"
        
        
    # neighborhoods
        try:
            neighborhoods_E = soup.select('dd.neighborhoods > span > a')
            neighborhoods = ""
            for N in neighborhoods_E:
                neighborhoods+= N.text + ", "
            neighborhoods = neighborhoods.strip(', ')
        except:
            neighborhoods = "Not Listed"
        
        
    #extra phones
        try:
            extra_phones_E = soup.select('dd.extra-phones > P')
            extra_phones = ""
            for P in extra_phones_E:
                extra_phones += P.text.replace('\xa0','') + "\n"
            extra_phones = extra_phones.strip('\n')
        except:
            extra_phones= "Not Listed"
        
    # emails
        try:
            email_E = soup.select('a[class="email-business"]')
            email = ""
            for M in email_E:
                email += 'http://www.'+ (M['href'].split(":")[1]) + "\n"
            email = email.strip('\n')
        except:
            email= "Not Listed"
        
        
    # other links
        try:
            other_links_E = soup.select('a[class="other-links"]')
            other_links = ""
            for L in other_links_E:
                other_links += L['href'] + "\n"
            other_links = other_links.strip('\n')
        except:
            other_links= "Not Listed"
        

    # social links
        try:
            social_links_E = soup.select('dd.social-links')[0].find_all('a')
            social_links = ""
            for S in social_links_E:
                social_links += S['href'] + "\n"
            social_links = social_links.strip('\n')
        except:
            social_links= "Not Listed"
        
        B_contacts = {
                    "Catagories": catagories,
                    "Neighborhoods": neighborhoods,
                    "Extra Phones": extra_phones,
                    "e-mail": email, 
                    "Other Links": other_links,
                    "Social Media": social_links
                    }
        
        for key, value in B_contacts.items():
            if value == '':
                B_contacts[key] = "Not Listed"
                        
        contacts.append(B_contacts)
        profile_no += 1
        
        if len(contacts) == 30: # saving the outputs to a csv file every 30 page
            results_scraped += len(contacts)
            save_index += 1
            
            s_df = pd.DataFrame(contacts)
            s_df.to_csv('outputs/' + f"{site_name} {search_subject} in {search_location} {save_index} secondary.csv", index= False)  
            
            contacts = []
            
            with open('outputs/' + txt_tracker,'a') as file:
                file.write(
                    '\n' +
                    f"{site_name} {search_subject} in {search_location} {save_index} secondary.csv"+
                    '\n'+
                    F'{profile_no}'
                )  
            
            
        previous_page = profile
        sleep(0.5) 
        
    if len(contacts) > 0: # saving the remaining output if the scraping is ended and the no of pages scraped is less than 30
        s_df0 = pd.DataFrame(contacts)
        s_df0.to_csv('outputs/' + f"{site_name} {search_subject} in {search_location} 0 secondary.csv", index= False)
        file = open('outputs/' + txt_tracker,'a')
        file.write('\n')
        file.write (f"{site_name} {search_subject} in {search_location} 0 secondary.csv") 
        file.write('\n')   
        file.close()
else:
    print("secondary file is already been scraped and saved")

Secondary Stage: 100%|█████████████████████████████████████████████████| 151/151 [18:22<00:00,  7.30s/profile]


Concatenating The Individually Saved Secondary csv Files Into One File. 

In [15]:

   
try:#checking if there was SECONDARY file already saved
    with open('outputs/' + txt_tracker,'r') as file:
        contents = file.read()
        P_csv_file = re.search('\w+SECONDARY\w+.csv',contents.replace(' ', '_'))[0].replace('_', ' ')
        m_df = pd.read_csv('outputs/' + P_csv_file) 
        
except:
    
    with open('outputs/' + txt_tracker,'r') as file:
        lines = file.readlines()  
        for line in lines[len(lines)::-1]: 
            if  f'{save_index} secondary.csv' in line:
                s_df0 = pd.read_csv('outputs/' + str(line).strip('\n'))
            
            elif 'secondary.csv' in line:
                s_df = pd.concat([s_df0, pd.read_csv('outputs/' + str(line).strip('\n'))], axis=0, ignore_index= True)
                s_df0 = s_df

            elif 'primary.csv' in line: 
                break

secondary_df = s_df 

In [16]:
secondary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Catagories     150 non-null    object
 1   Neighborhoods  150 non-null    object
 2   Extra Phones   150 non-null    object
 3   e-mail         150 non-null    object
 4   Other Links    150 non-null    object
 5   Social Media   150 non-null    object
dtypes: object(6)
memory usage: 7.2+ KB


In [17]:
secondary_df.tail()

,Catagories,Neighborhoods,Extra Phones,e-mail,Other Links,Social Media
145,"New Car Dealers, Auto Repair & Service, Automo...",Not Listed,Not Listed,http://www.teamnissan@teamnissandirect.com,http://www.teamnissandirect.com,https://facebook.com/teamnissanoxnard\nteamnis...
146,"New Car Dealers, Auto Oil & Lube, Auto Repair ...","Downtown Campbell, Central Campbell",TollFree:(800) 367-3872,Not Listed,http://local.firestonecompleteautocare.com/cal...,https://www.facebook.com/FirestoneCompleteAuto...
147,"New Car Dealers, Auto Body Parts, Auto Repair ...","Central Alameda, South LA",Not Listed,http://www.ajp2207@gmail.com,https://www.aplusfordandchevy.com,Not Listed
148,"New Car Dealers, Auto Repair & Service, Automo...",Not Listed,Not Listed,http://www.slowery@corningford.com\nhttp://www...,http://www.redbluffdodgechryslerjeepram.com,Not Listed
149,"New Car Dealers, New Truck Dealers",Not Listed,Not Listed,http://www.sales@corningford.com\nhttp://www.s...,http://www.corningford.com,https://www.facebook.com/CorningFord/timeline


In [18]:
secondary_df.to_csv('outputs/' + f"{site_name} SECONDARY {search_subject} in {search_location}.csv", index= False)

with open('outputs/' + txt_tracker,'a') as file:         
    file.write('\n')
    file.write (f"{site_name} SECONDARY {search_subject} in {search_location}.csv") 

In [19]:
# deleting the secondary files after saving the concatenated SECONDARY file
files = os.listdir('outputs/')
for file in files:
    if 'secondary' in file:
        os.remove('outputs/' + file)

Joining The secondary_df With the main_df To Get The Final Result 

In [20]:
df_combined = main_df.join(secondary_df)
df_combined.index += 1  # make the index start from one

In [21]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 1 to 151
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Business Name  151 non-null    object 
 1   Profile        151 non-null    object 
 2   Phone Number   151 non-null    object 
 3   Address        151 non-null    object 
 4   Website        151 non-null    object 
 5   Rating         151 non-null    float64
 6   Review Count   151 non-null    int64  
 7   Catagories     150 non-null    object 
 8   Neighborhoods  150 non-null    object 
 9   Extra Phones   150 non-null    object 
 10  e-mail         150 non-null    object 
 11  Other Links    150 non-null    object 
 12  Social Media   150 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 15.5+ KB


In [22]:
df_combined.head()

,Business Name,Profile,Phone Number,Address,Website,Rating,Review Count,Catagories,Neighborhoods,Extra Phones,e-mail,Other Links,Social Media
1,LemonBuyBack.com-Law Offices of Jon Jacobs,https://www.yellowpages.com/nationwide/mip/lem...,Not Listed,Not Available,Not Available,0.0,0,New Car Dealers,"Chesterfield Square, Congress Central, South L...",Phone:(323) 293-6850\nPhone:(323) 293-0977\nFa...,http://www.info@lincolnucc.org,http://www.lincolnucc.org\nhttps://lincolnucc....,Not Listed
2,LemonBuyBack.com-Law Offices of Jon Jacobs,https://www.yellowpages.com/nationwide/mip/lem...,(916) 626-4164,Not Available,http://lemonbuyback.com,5.0,10,"Automobile Body Repairing & Painting, Auto Rep...","North Hollywood North East, South Valley",Not Listed,http://www.G.David@autobahncenter.com,http://www.autobahncenter.com,Not Listed
3,El Cajon Ford,https://www.yellowpages.com/el-cajon-ca/mip/el...,(619) 333-8825,"1595 E Main St, El Cajon, CA 92021",http://www.elcajonford.com,5.0,43,"Automobile Parts & Supplies, Auto Repair & Ser...","Hollywood Hills West, Central LA",Phone:(310) 271-3476\nPhone:(310) 274-3476\nPh...,Not Listed,http://www.universalcitynissan.com,Not Listed
4,Jaguar Coventry Cars of San Diego,https://www.yellowpages.com/san-diego-ca/mip/j...,(619) 297-9393,"5097 Santa Fe St, San Diego, CA 92109",http://www.coventryjaguar.com,4.5,3,"Automobile & Truck Brokers, Automobile Parts &...",Rosemead,Not Listed,Not Listed,Not Listed,Not Listed
5,Allied Auto Works,https://www.yellowpages.com/los-altos-ca/mip/a...,(650) 209-6897,"2073 Grant Rd, Los Altos, CA 94024",http://www.alliedautoworks.com,0.0,0,New Car Dealers,Tustin,Not Listed,Not Listed,Not Listed,Not Listed


> ### Saving The Completed DataFrame df_combined & Deleting The Main File & The Secondary File As They No Longer Needed

In [23]:
df_combined.to_csv('outputs/' + f"{site_name} {search_subject} in {search_location}.csv", index= False)

In [24]:
# deleting both the PRIMARY and SECONDARY files after saving the joined end result
files = os.listdir('outputs/')
for file in files:
    if 'PRIMARY' in file or 'SECONDARY' in file or file == txt_tracker:
        os.remove('outputs/' + file)